## *LAB 3  Practicing Extraction in ETL*

### *Historical Description of the Car Sales Dataset*
- *This dataset represents simulated used car sales data  used car sales data over two months (April–May 2025). It includes purchases by major automotive dealers, rental companies, and vehicle auction services such as AutoNation, CarMax, Enterprise Holdings, and Manheim.*

- *The dataset features popular car brands like Toyota, Honda, Ford, Hyundai, and Chevrolet, with model years ranging from 2015 to 2023. It records key details such as vehicle age, mileage, price, and payment types, offering a realistic view of market diversity and buyer preferences.*


#### *Key Attributes*

*`Date of Sale:` The date on which the car transaction occurred, showing when the vehicle was purchased.*

*`Dealer:` Represents the company or automotive group that sold or distributed the vehicle.*

*`Car Make and Model:` Describes the manufacturer (e.g., Toyota, Ford) and specific model of the vehicle, indicating brand preference and market variety.*

*`Manufacture Year:` Indicates the year the vehicle was made, providing context for its age and potential value depreciation.*

*`Odometer Reading:` Shows the total distance the vehicle had traveled before the sale, an important factor influencing its condition and price.*

*`Sale Price:` The amount (in USD) for which the vehicle was sold, indicating market demand and valuation.*

*`Payment Method:` Specifies how the transaction was completed—via cash, credit, or loan—reflecting diverse financing choices.*

*`Last Updated:` The timestamp of the most recent update to the record, helping track edits or corrections.*

#### Import libraries

In [475]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

### `STEP 1: Generate synthetic data`

In [476]:
# Generate Car Sales Dataset
dealers = ['AutoNation', 'CarMax', 'Penske Automotive', 'Lithia Motors', 'Sonic Automotive', 
           'Enterprise Holdings', 'Hertz Global Holdings', 'Avis Budget Group', 'Manheim', 'Copart']

car_inventory = {
    'Toyota': ['Corolla', 'Camry', 'RAV4'],
    'Honda': ['Civic', 'Accord', 'CR-V'],
    'Ford': ['Focus', 'Fusion', 'Escape'],
    'Hyundai': ['Elantra', 'Tucson', 'Santa Fe'],
    'Chevrolet': ['Malibu', 'Cruze', 'Equinox']
}

payment_methods = ['Cash', 'Credit', 'Loan']

sales_records = []
start_date = datetime(2025, 4, 1)
for day_offset in range(1, 61):
    current_date = start_date + timedelta(days=day_offset)
    for _ in range(random.randint(3, 6)):
        make = random.choice(list(car_inventory.keys()))
        model = random.choice(car_inventory[make])
        manufacture_year = random.randint(2015, 2023)
        odometer_reading = random.randint(10000, 120000)
        sale_price = random.randint(5000, 25000)

        # Introduce missing values with 10% chance each
        customer = random.choice(dealers)
        if random.random() < 0.10:
            customer = None

        mileage = odometer_reading
        if random.random() < 0.10:
            mileage = None

        payment_type = random.choice(payment_methods)
        if random.random() < 0.10:
            payment_type = None

        sales_records.append({
            'id': random.randint(10000, 99999),
            'customer': customer,
            'date': current_date.date().isoformat(),
            'car_make': make,
            'car_model': model,
            'year': manufacture_year,
            'mileage': mileage,
            'price': sale_price,
            'payment_type': payment_type,
            'last_updated': (current_date + timedelta(hours=random.randint(0, 23),
                                                      minutes=random.randint(0, 59))).isoformat()
        })


####  Create DataFrame

In [477]:
# Create DataFrame
df = pd.DataFrame(sales_records )

# Save to CSV
df.to_csv('car_sales_data_2025.csv', index=False)

# Display the first 10 records
df.head(10)

,id,customer,date,car_make,car_model,year,mileage,price,payment_type,last_updated
0,34711,CarMax,2025-04-02,Ford,Escape,2023,111613.0,5335,Cash,2025-04-02T16:56:00
1,98937,AutoNation,2025-04-02,Hyundai,Santa Fe,2023,70643.0,19719,Credit,2025-04-02T07:25:00
2,79192,Copart,2025-04-02,Hyundai,Tucson,2022,71863.0,17754,Cash,2025-04-02T00:31:00
3,74489,None,2025-04-03,Honda,Accord,2019,85494.0,18033,Credit,2025-04-03T10:40:00
4,35537,Copart,2025-04-03,Toyota,RAV4,2021,15757.0,15491,Credit,2025-04-03T17:38:00
5,36051,Hertz Global Holdings,2025-04-03,Toyota,Camry,2017,78079.0,19646,Credit,2025-04-03T09:37:00
6,41675,Penske Automotive,2025-04-03,Ford,Fusion,2019,118027.0,17536,Loan,2025-04-03T12:19:00
7,12382,AutoNation,2025-04-03,Honda,CR-V,2015,45492.0,7349,Cash,2025-04-03T02:41:00
8,72212,Avis Budget Group,2025-04-03,Hyundai,Santa Fe,2019,21428.0,14731,Cash,2025-04-03T08:01:00
9,94424,Enterprise Holdings,2025-04-04,Ford,Fusion,2022,82589.0,14272,Loan,2025-04-04T22:27:00


In [478]:
# Count number of unique days
unique_days = df['date'].nunique()
print(f"Number of unique days with sales records: {unique_days}")

Number of unique days with sales records: 60


 ###  `Section 1: Full Extraction`

**Full Extraction** *means retrieving the entire dataset from the data source every time the extraction process runs, without considering whether any data has changed since the last extraction. This approach ensures that you always have a complete and up-to-date copy of the dataset.*

####  `in the following code:`
- *The entire dataset is read from the source "car_sales_data_may_2025.csv".* 
- *All records, regardless of whether they are new, updated, or unchanged, are loaded into memory.* 
- *Basic information about the dataset (such as the number of rows and columns) and a sample of the data are optionally displayed to help verify the extraction and understand the data structure.*


In [479]:
# FULL EXTRACTION:Load entire dataset
df_full = pd.read_csv("car_sales_data_2025.csv", parse_dates=["last_updated"])
print(f"Pulled {len(df_full)} rows via full extraction.")

# Show basic dataset info 
print(f"Dataset shape: {df_full.shape}") 

Pulled 269 rows via full extraction.
Dataset shape: (269, 10)


### `Sample_Data`

In [480]:
print("Columns:", df_full.columns.tolist())

Columns: ['id', 'customer', 'date', 'car_make', 'car_model', 'year', 'mileage', 'price', 'payment_type', 'last_updated']



- *A sample of the first few rows is printed to verify the data content and structure.*

In [481]:
df_full.head(5)

,id,customer,date,car_make,car_model,year,mileage,price,payment_type,last_updated
0,34711,CarMax,2025-04-02,Ford,Escape,2023,111613.0,5335,Cash,2025-04-02 16:56:00
1,98937,AutoNation,2025-04-02,Hyundai,Santa Fe,2023,70643.0,19719,Credit,2025-04-02 07:25:00
2,79192,Copart,2025-04-02,Hyundai,Tucson,2022,71863.0,17754,Cash,2025-04-02 00:31:00
3,74489,NaN,2025-04-03,Honda,Accord,2019,85494.0,18033,Credit,2025-04-03 10:40:00
4,35537,Copart,2025-04-03,Toyota,RAV4,2021,15757.0,15491,Credit,2025-04-03 17:38:00


In [482]:
#full_extraction_ containing the results as csv file
df.to_csv('car_sales_full_extraction_output.csv', index=False)

# Save incremental extraction results to CSV
print("Incremental extraction output saved as 'car_sales_full_extraction_output.csv")


Incremental extraction output saved as 'car_sales_full_extraction_output.csv


### `Section 2: Incremental Extraction`

-*`The steps are as follows:`*

- *The last extraction timestamp is read from the file `last_extraction.txt`*. 
- *The dataset is loaded from the file `car_sales_data_may_2025.csv`.  
   The `last_updated` column is parsed as a datetime object to facilitate time-based comparisons.*

- *The timestamp obtained from the text file is converted into a `pandas`    datetime object.*  
- *This allows accurate filtering of records based on their update time.*

- *The dataset is filtered to include only the rows where the `last_updated`   value is later than the last extraction timestamp.*  
- *This step simulates **incremental extraction** by retrieving only the new or modified records.*

- *A sample of these new or updated records is shown to verify the verification of the extraction process.*


#### `1. Create the tracking file: last_extraction.txt`

- *This file stores the timestamp of the last extraction, so the incremental process knows where to pick up.*



In [483]:
with open("last_extraction.txt", "w") as f:
    f.write("2025-04-20 12:00:00")

#### `2.Perform Incremental Extraction`
- *Read the last extraction timestamp from the tracking file.*

In [484]:
# INCREMENTAL EXTRACTION
with open("last_extraction.txt", "r") as f:
    last_extraction = f.read().strip()

#### *Load full dataset with date parsing and Converting to datetime*

In [485]:
df = pd.read_csv("car_sales_data_2025.csv", parse_dates=["last_updated"])
last_extraction_time = pd.to_datetime(last_extraction)

####  *Filter new or updated records since last extraction*

In [486]:
df_incremental = df[df['last_updated'] > last_extraction_time]
print(f"Pulled {len(df_incremental)} new/updated rows since {last_extraction}.")
df_incremental.head()

Pulled 182 new/updated rows since 2025-04-20 12:00:00.


,id,customer,date,car_make,car_model,year,mileage,price,payment_type,last_updated
85,85049,AutoNation,2025-04-20,Honda,Accord,2021,80508.0,10552,Credit,2025-04-20 23:49:00
86,92102,Penske Automotive,2025-04-20,Honda,Civic,2016,56317.0,15748,Cash,2025-04-20 16:48:00
87,25058,Enterprise Holdings,2025-04-20,Honda,Accord,2015,92961.0,5691,Cash,2025-04-20 14:11:00
88,63280,Lithia Motors,2025-04-20,Honda,Accord,2016,95518.0,20580,Cash,2025-04-20 19:53:00
91,93218,AutoNation,2025-04-21,Honda,Accord,2018,114797.0,5329,Cash,2025-04-21 00:23:00


In [487]:
# Save incremental extraction results to CSV
df_incremental.to_csv('car_sales_incremental_extraction_output.csv', index=False)
print("Incremental extraction output saved as 'incremental_extraction_output.csv'.")

Incremental extraction output saved as 'incremental_extraction_output.csv'.


### `Update the last_extraction.txt`

#### *Incremental Extraction*

- *First,  keep a note of the last time we checked for new data by saving a date and time in a file called `last_extraction.txt`.*
- *Next, look through the full data but only pick out the new or updated entries that happened after that saved date and time.*
- *Then show a message telling how many new or updated records we found since the last time we checked.*

### *Save New Timestamp*

- *After getting the new data, we update the saved date and time in `last_extraction.txt` to mark the most recent check.*
- *This helps us know where to start from next time so we only get new changes, not old data again.*


In [488]:
# Get the most recent update
new_checkpoint = df['last_updated'].max()
# Save it
with open("last_extraction.txt", "w") as f:
    f.write(new_checkpoint.isoformat())
print(f"Updated last_extraction.txt to {new_checkpoint}")

Updated last_extraction.txt to 2025-05-31 19:12:00


## *LAB 4: Transform in ETL*

### Transform Full Data
*Full Transformation applies all cleaning, enrichment, and formatting operations to the entire dataset,  — regardless of whether the  records are new, updated, or unchanged.*

*`Use Cases`:Ideal for initial data loads, schema updates, and ensuring consistency across the entire dataset.*

*`Pros`:It ensures complete uniformity, is easier to debug, and supports accurate analytics and reporting.*

*`Cons`:It is slower, uses more resources, and can be redundant if no changes exist in the data.*


#### *Data Cleaning*

In [489]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269 entries, 0 to 268
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            269 non-null    int64         
 1   customer      238 non-null    object        
 2   date          269 non-null    object        
 3   car_make      269 non-null    object        
 4   car_model     269 non-null    object        
 5   year          269 non-null    int64         
 6   mileage       241 non-null    float64       
 7   price         269 non-null    int64         
 8   payment_type  252 non-null    object        
 9   last_updated  269 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 21.1+ KB


In [490]:
df_full.duplicated().sum()

0

####  *Handle missing values*

In [491]:
df_full.isnull().sum()

id               0
customer        31
date             0
car_make         0
car_model        0
year             0
mileage         28
price            0
payment_type    17
last_updated     0
dtype: int64

In [492]:
# fill string-based NaNs with a placeholder
df_full['customer'] = df_full['customer'].fillna('Unknown')
df_full['payment_type'] = df_full['payment_type'].fillna('Unknown')

### *Group-wise Imputation for mileage*

In [493]:
# Fill missing mileage with group mean (based on car_make and car_model)
df_full['mileage'] = df_full.groupby(['car_make', 'car_model'])['mileage']\
                            .transform(lambda x: x.fillna(x.mean()))

#### *Categorize mileage*

In [494]:
df_full['mileage_bin'] = pd.cut(
    df_full['mileage'],
    bins=[0, 30000, 60000, 90000, 120000],
    labels=['Low', 'Medium', 'High', 'Very High']
)

In [495]:
# Group mileage by bin and get range with explicit 'observed=True'
mileage_bin = df_full.groupby('mileage_bin', observed=True)['mileage'].agg(
    Range=lambda x: f"{int(x.min())}–{int(x.max())}"
).reset_index()

# Reorder columns for clarity
mileage_bin = mileage_bin[['Range', 'mileage_bin']]

# Display the result
mileage_bin


,Range,mileage_bin
0,10642–29138,Low
1,31187–59562,Medium
2,60782–89742,High
3,90817–119011,Very High


#### *Impute Missing price Values Based on Groups*


In [496]:
df_full['price'] = df_full.groupby(['car_make', 'car_model', 'mileage_bin'], observed=True)['price']\
                          .transform(lambda x: x.fillna(x.mean()))

### *Data Cleaning Summary*

#### *`String-Based Missing Value Handling`*  
Missing string values in the `customer` and `payment_type  ` column were filled with the placeholder `"Unknown"` to ensure consistency and prevent issues in downstream processing.

####  *`Group-wise Imputation for mileage`*  
Missing values in the `mileage` column were imputed using the mean mileage grouped by `car_make` and `car_model`, offering a more accurate estimate than a  average.

#### *`Categorize mileage`*  
The `mileage` column was binned into defined ranges (`Low`, `Medium`, `High`, `Very High`) to simplify analysis and assist with grouped imputations.

#### *`Impute Missing price Values Based on Groups`*  
Missing `price` values were filled using the mean price calculated within groups defined by `car_make`, `car_model`, and `mileage_bin`, ensuring context-aware imputation.


### *Enrichment* 

In [497]:
# Add vehicle_age column
df_full['vehicle_age'] = 2025 - df_full['year']
# Print a more meaningful preview
print("Vehicle Age:")
print(df_full[['car_make', 'car_model', 'year', 'vehicle_age']].head())

Vehicle Age:
  car_make car_model  year  vehicle_age
0     Ford    Escape  2023            2
1  Hyundai  Santa Fe  2023            2
2  Hyundai    Tucson  2022            3
3    Honda    Accord  2019            6
4   Toyota      RAV4  2021            4


#### *Importance  Enrichment*

*This section introduces new features derived from existing columns to add analytical value and support deeper insights:*

#### *`vehicle_age `*
*This calculates how old each car is. Instead of working with the raw `year`, `vehicle_age` provides a clear numeric representation that is easier to compare and analyze.*




#### *Key Restructuring*

In [498]:
df_full.drop(columns='id', inplace=True)

# Add a new surrogate key as 'patient_id'
df_full.insert(0, 'sales_id', range(1, len(df_full) + 1))

# Preview the result
df_full.head()

,sales_id,customer,date,car_make,car_model,year,mileage,price,payment_type,last_updated,mileage_bin,vehicle_age
0,1,CarMax,2025-04-02,Ford,Escape,2023,111613.0,5335,Cash,2025-04-02 16:56:00,Very High,2
1,2,AutoNation,2025-04-02,Hyundai,Santa Fe,2023,70643.0,19719,Credit,2025-04-02 07:25:00,High,2
2,3,Copart,2025-04-02,Hyundai,Tucson,2022,71863.0,17754,Cash,2025-04-02 00:31:00,High,3
3,4,Unknown,2025-04-03,Honda,Accord,2019,85494.0,18033,Credit,2025-04-03 10:40:00,High,6
4,5,Copart,2025-04-03,Toyota,RAV4,2021,15757.0,15491,Credit,2025-04-03 17:38:00,Low,4


#### *Importance of Introducing Surrogate Key `sales_id`*

To enhance record identification and maintain consistency, the original `id` column was removed, and a new surrogate key named `sales_id` was added. This key uniquely identifies each record and was inserted as the first column in the dataset.

*Using `sales_id` as a surrogate key improves data integrity, simplifies referencing, and supports structured data processing.*

--- 
*A surrogate key like `sales_id` guarantees that each row has a unique identifier, which is critical for maintaining data integrity—especially if the original `id` column contained duplicates or gaps.*


*As Surrogate keys simplify merging and integrating with other datasets by providing a clean, consistent identifier.*


In [499]:
df_full.to_csv("transformed_full.csv", index=False)
print("Full transformed dataset has been successfully saved as 'transformed_full.csv'.")

Full transformed dataset has been successfully saved as 'transformed_full.csv'.


### Transform Incremental Data

*Incremental Transformation targets only newly added or changed data 
(often based on timestamps or IDs) and applies transformation logic selectively.*

*`Use Cases`:Best for frequent updates in production, such as daily loads or streaming data.*

*`Pros`:  It is faster, more efficient, and reduces computation by only processing changed data.*


*`Cons`:It requires change tracking, adds complexity, and may lead to inconsistencies if historical transformation logic changes.*


#### *Data Cleaning*

In [501]:
#Load dataset with incremental updates
df_incremental= pd.read_csv("car_sales_incremental_extraction_output.csv", parse_dates=["last_updated"])

In [502]:
df_incremental.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182 entries, 0 to 181
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            182 non-null    int64         
 1   customer      162 non-null    object        
 2   date          182 non-null    object        
 3   car_make      182 non-null    object        
 4   car_model     182 non-null    object        
 5   year          182 non-null    int64         
 6   mileage       163 non-null    float64       
 7   price         182 non-null    int64         
 8   payment_type  171 non-null    object        
 9   last_updated  182 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 14.3+ KB


In [503]:
df_incremental.duplicated().sum()

0

In [504]:
df_incremental.isnull().sum()

id               0
customer        20
date             0
car_make         0
car_model        0
year             0
mileage         19
price            0
payment_type    11
last_updated     0
dtype: int64

In [505]:
# fill string-based NaNs with a placeholder
df_incremental['customer'] = df_incremental['customer'].fillna('Unknown')
df_incremental['payment_type'] = df_incremental['payment_type'].fillna('Unknown')

### *Group-wise Imputation for mileage*

In [506]:
# Fill missing mileage with group mean (based on car_make and car_model)
df_full['mileage'] = df_incremental.groupby(['car_make', 'car_model'])['mileage']\
                            .transform(lambda x: x.fillna(x.mean()))

#### *Categorize mileage*

In [507]:
df_incremental['mileage_bin'] = pd.cut(
    df_incremental['mileage'],
    bins=[0, 30000, 60000, 90000, 120000],
    labels=['Low', 'Medium', 'High', 'Very High']
)

In [508]:
# Group mileage by bin and get range with explicit 'observed=True'
mileage_bin = df_incremental.groupby('mileage_bin', observed=True)['mileage'].agg(
    Range=lambda x: f"{int(x.min())}–{int(x.max())}"
).reset_index()

# Reorder columns for clarity
mileage_bin = mileage_bin[['Range', 'mileage_bin']]

# Display the result
mileage_bin


,Range,mileage_bin
0,10642–27969,Low
1,31187–59547,Medium
2,60782–89742,High
3,90817–119011,Very High


#### *Impute Missing price Values Based on Groups*


In [509]:
df_full['price'] = df_incremental.groupby(['car_make', 'car_model', 'mileage_bin'], observed=True)['price']\
                          .transform(lambda x: x.fillna(x.mean()))

### *Enrichment* 

In [510]:
# Add vehicle_age column
df_incremental['vehicle_age'] = 2025 - df_incremental['year']
# Print a more meaningful preview
print("Vehicle Age:")
print(df_incremental[['car_make', 'car_model', 'year', 'vehicle_age']].head())

Vehicle Age:
  car_make car_model  year  vehicle_age
0    Honda    Accord  2021            4
1    Honda     Civic  2016            9
2    Honda    Accord  2015           10
3    Honda    Accord  2016            9
4    Honda    Accord  2018            7


#### *Key Restructuring*

In [511]:
# Add a new surrogate key as 'patient_id'
df_incremental.insert(0, 'sales_id', range(1, len(df_incremental) + 1))

df_incremental.drop(columns='id', inplace=True)

# Preview result
df_incremental.head()

,sales_id,customer,date,car_make,car_model,year,mileage,price,payment_type,last_updated,mileage_bin,vehicle_age
0,1,AutoNation,2025-04-20,Honda,Accord,2021,80508.0,10552,Credit,2025-04-20 23:49:00,High,4
1,2,Penske Automotive,2025-04-20,Honda,Civic,2016,56317.0,15748,Cash,2025-04-20 16:48:00,Medium,9
2,3,Enterprise Holdings,2025-04-20,Honda,Accord,2015,92961.0,5691,Cash,2025-04-20 14:11:00,Very High,10
3,4,Lithia Motors,2025-04-20,Honda,Accord,2016,95518.0,20580,Cash,2025-04-20 19:53:00,Very High,9
4,5,AutoNation,2025-04-21,Honda,Accord,2018,114797.0,5329,Cash,2025-04-21 00:23:00,Very High,7


In [512]:
df_full.to_csv("transformed_incremental.csv.csv", index=False)
print("Incremental transformed dataset has been successfully saved as 'transformed_full.csv'.")

Incremental transformed dataset has been successfully saved as 'transformed_full.csv'.
